# Tarea 8: 

Solución de los ejercicios de la sección `Intervalos de confianza y remuestreo` del libro del curso [Fundamentos Estadísticos](https://tereom.github.io/fundamentos-2022/intervalos-de-confianza-y-remuestreo.html)

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
path_universal = "C:\\Users\\Miguel\\Documents\\Github\\itam-mcd"
os.listdir(path_universal)

['.git',
 '.gitignore',
 'aprendizaje-automatico',
 'estadistica-computacional',
 'fundamentos-estadisticos',
 'mineria-analisis-datos',
 'optimizacion',
 'README.md']

In [3]:
path_asignatura = "fundamentos-estadisticos"
path_asignatura = os.path.join(path_universal, path_asignatura)
os.listdir(path_asignatura)

['datos', 'libros', 'README.md']

## ENIGH

Para este ejercicio usaremos los datos de la [ENIGH 2014](https://www.inegi.org.mx/programas/enigh/tradicional/2014/). En particular las variables alimentos, vestido, vivienda, salud, comunica, educacion y esparci (esparcimiento) que indican el gasto trimestral en cada una de las categorías. 
1. Calcula los deciles de ingreso usando la variable de ingreso corriente (ing_cor).

>Debes tomar en cuenta el diseño de la muestra, puedes usar la función `survey_quantile()` del paquete `srvyr` o `svyquantile()` del paquete `survey`. Reporta las estimaciones y sus errores estándar usando el bootstrap de Rao y Wu.

In [4]:
#Se obtiene dataset original
file_path = "datos\\concentradohogar.csv"

columns = ['folioviv', 'foliohog', 'est_dis', 'upm', 'factor_hog', 'ing_cor', 'alimentos',
           'vestido', 'vivienda', 'salud', 'transporte', 'comunica', 'educacion', 'esparci']
concentrado_hogar = pd.read_csv(os.path.join(path_asignatura, file_path))
concentrado_hogar = concentrado_hogar[columns]
concentrado_hogar['folio_unico'] = concentrado_hogar['folioviv'].astype(str)+'_'+concentrado_hogar['foliohog'].astype(str)

concentrado_hogar

,folioviv,foliohog,est_dis,upm,factor_hog,ing_cor,alimentos,vestido,vivienda,salud,transporte,comunica,educacion,esparci,folio_unico
0,100008302,1,5,670,694,39786.79,11172.63,0.00,2199.32,1401.83,5490.71,1110.00,0.00,900.00,100008302_1
1,100008303,1,5,670,694,19523.90,1941.38,0.00,0.00,117.38,0.00,0.00,0.00,0.00,100008303_1
2,100008304,1,5,670,694,99257.66,16759.08,489.12,2989.43,1242.37,5829.25,2540.32,0.00,0.00,100008304_1
3,100008305,1,5,670,694,87883.68,8678.43,293.47,1100.00,420.63,1440.00,1440.00,3019.35,0.00,100008305_1
4,100010401,1,5,600,660,84427.04,12873.09,0.00,3062.50,0.00,7006.44,1200.00,0.00,1050.00,100010401_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19474,3260733806,1,197,26180,700,19388.37,925.70,0.00,444.00,1888.01,0.00,0.00,0.00,450.00,3260733806_1
19475,3260733807,1,197,26180,700,19089.50,7868.43,0.00,1784.89,3076.60,2749.74,435.48,0.00,0.00,3260733807_1
19476,3260733808,1,197,26180,700,11590.15,1902.82,0.00,1718.54,0.00,976.51,609.67,0.00,29.03,3260733808_1
19477,3260733809,1,197,26180,700,16412.44,2256.34,0.00,653.70,782.60,1260.55,0.00,0.00,0.00,3260733809_1


**Calculando** los deciles, primero se cambia la estructura del conjunto de datos, de `DataFrame` $\rightarrow$ `dict`

In [5]:
#Se genera un diccionario con la información del df
con_hog_dict = concentrado_hogar.to_dict()

#Se descartan columnas de folio que ya están contenidas en folio_unico
con_hog_dict.pop('folioviv')
con_hog_dict.pop('foliohog')

#Se crea llave para almacenar ing_cor expandida
con_hog_dict['expand_ing_cor'] = {}

con_hog_dict.keys()

dict_keys(['est_dis', 'upm', 'factor_hog', 'ing_cor', 'alimentos', 'vestido', 'vivienda', 'salud', 'transporte', 'comunica', 'educacion', 'esparci', 'folio_unico', 'expand_ing_cor'])

Ahora, se **expande** la variable `ing_cor` por el número definido en la variable `factor_hog` para almacenarlo en el diccionario

In [6]:
%%time
for idx in con_hog_dict['folio_unico'].keys():
    expand = np.repeat(con_hog_dict['ing_cor'][idx], con_hog_dict['factor_hog'][idx])
    
    con_hog_dict['expand_ing_cor'][idx] = expand

Wall time: 210 ms


In [7]:
concentrado_hogar['factor_hog'].value_counts()

1975    61
332     59
339     57
1379    56
284     51
        ..
4010     1
2432     1
5651     1
2148     1
4292     1
Name: factor_hog, Length: 1837, dtype: int64

2. Crea una nueva variable que indique el decil de ingreso para cada hogar. 

>**Tips**:
>1) una función que puede resultar útil es `cut2()` (de `Hmisc`), 
>2) si usas el paquete `srvyr` puedes usar `mutate()` sobre el objeto `survey` con pesos de replicaciones bootstrap.